In [5]:
from IPython.core.magic import register_cell_magic, register_line_magic

@register_cell_magic
def sqlite3(line, cell):
    cell = [l for l in cell.split('\n') if l[0:2] != '--' and l[0:2] not in ['--', '##']]
    cell = '\n'.join(cell)
    return cur.execute(cell).fetchall()

@register_cell_magic
def sqlite3script(line, cell):
    cell = [l for l in cell.split('\n') if len(l) >= 2 and l[0:2] not in ['--', '##'] ]
    cell = '\n'.join(cell)
    conn.executescript(cell)
    conn.commit()

@register_line_magic
def sql(line):
    line = [l for l in line.split('\n') if l[0:2] != '--' and l[0:2] not in ['--', '##']]
    line = '\n'.join(line)
    return cur.execute(line).fetchall()

In [6]:
import sqlite3

conn = sqlite3.connect(':memory:')
cur = conn.cursor()

In [7]:
##
## Creación de las tablas
##
sqlcmd = """
CREATE TABLE tbl0 (
    K0  CHAR(1),
    c01 INT,
    c02 INT,
    c03 CHAR(4),
    c04 FLOAT
    );
"""
cur.execute(sqlcmd).fetchall()

[]

In [8]:
##

sqlcmd = """
CREATE TABLE tbl1 (
    K0  CHAR(1),
    K1  INT,
    c12 FLOAT,
    c13 INT,
    c14 DATE,
    c15 FLOAT,
    c16 CHAR(4)
    );
"""
cur.execute(sqlcmd).fetchall()

[]

In [9]:
##

sqlcmd = """
CREATE TABLE tbl2 (
    K1  INT,
    c21 FLOAT,
    c22 INT,
    c23 DATE,
    c24 FLOAT,
    c25 CHAR(5)
    );
"""
cur.execute(sqlcmd).fetchall()

[]

In [11]:
##
## Carga de datos
##
text =  open('tbl0.csv', 'rt', encoding='utf-8').readlines()
text = [line[:-1] if line[-1] == '\n' else line for line in text]    
text = [line.split(',') for line in text]
text = [tuple(line) for line in text]
cur.executemany('INSERT INTO tbl0 VALUES (?,?,?,?,?)', text)

In [12]:
text =  open('tbl1.csv', 'rt', encoding='utf-8').readlines()
text = [line[:-1] if line[-1] == '\n' else line for line in text]    
text = [line.split(',') for line in text]
text = [tuple(line) for line in text]
cur.executemany('INSERT INTO tbl1 VALUES (?,?,?,?,?,?,?)', text)

In [13]:
text =  open('tbl2.csv', 'rt', encoding='utf-8').readlines()
text = [line[:-1] if line[-1] == '\n' else line for line in text]    
text = [line.split(',') for line in text]
text = [tuple(line) for line in text]
cur.executemany('INSERT INTO tbl2 VALUES (?,?,?,?,?,?)', text)

### q01.sql

~~~
## 
## Escriba una consulta que devuelva la suma del campo c12
## de la tabla tbl1
## 
~~~

In [18]:
%%sqlite3
SELECT SUM(c12)
FROM tbl1;

[(15137.63,)]

### q02.sql

~~~
## 
## Escriba una consulta que retorne la cantidad de registros
## de la tabla tbl1
## 
~~~

In [20]:
%%sqlite3
SELECT COUNT(*)
FROM tbl1;

[(30,)]

### q03.sql

~~~
## 
## Escriba una consulta que retorne los primeros cinco
## registros de la tabla tbl1 ordenados por fecha
## 
~~~

In [22]:
%%sqlite3
SELECT *
FROM tbl1
ORDER BY c14 LIMIT 5;

[('A', 20, 938.16, 300, '2016-09-12', 0.19, 'BECB'),
 ('C', 15, 370.58, 900, '2016-10-01', 0.11, 'GCDD'),
 ('E', 22, 118.77, 900, '2016-10-29', 0.32, 'GEFE'),
 ('B', 12, 999.72, 800, '2016-11-09', 0.26, 'FCGD'),
 ('E', 14, 832.44, 800, '2016-11-22', 0.39, 'EGFD')]

### q04.sql

~~~
## 
## Escriba una consulta que retorne los campos K0 y c16
## para los registros de la tabla tbl1 para los que la 
## columna c16 empieza por la misma letra de la columna K0
## 
~~~

In [40]:
%%sqlite3
SELECT k0, c16
FROM tbl1
WHERE c16 LIKE k0 || '%';

[('E', 'EGFD'), ('B', 'BDEE'), ('C', 'CCCE')]

### q05.sql

~~~
## 
## Escriba una consulta que retorne todos los campos de 
## la tabla tbl0 para los que el campo c02 es igual a 100
## o igual a 600
## 
~~~

In [41]:
%%sqlite3
SELECT *
FROM tbl0
WHERE c02 IN (100, 600);

[('B', 7000, 100, 'OLPKN', 0.2),
 ('C', 1000, 600, 'LMMML', 0.2),
 ('D', 4000, 600, 'PJLJL', 0.4),
 ('G', 5000, 100, 'NLPLO', 0.2)]

### q06.sql

~~~
## 
## Escriba una consulta que retorne todos los campos de 
## la tabla tbl1 ordenada por fecha (c14) para los 
## registros con K0 igual a A
## 
~~~

In [44]:
%%sqlite3
SELECT *
FROM tbl1
WHERE k0 = 'A'
ORDER BY c14;

[('A', 20, 938.16, 300, '2016-09-12', 0.19, 'BECB'),
 ('A', 30, 135.8, 900, '2017-01-26', 0.23, 'EGAB'),
 ('A', 18, 142.99, 100, '2017-02-12', 0.48, 'GGFD'),
 ('A', 26, 456.47, 400, '2018-01-28', 0.11, 'FGED'),
 ('A', 6, 391.42, 300, '2018-05-15', 0.22, 'BFGB'),
 ('A', 10, 816.51, 600, '2019-04-25', 0.4, 'DAGC')]

### q07.sql

~~~
## 
## Escriba una consulta que retorne todos los campos de 
## la tabla tbl1 ordenada por fecha (c14) para los 
## registros con K0 diferente a A y B y c13 diferente 
## de 200 y 900
##
~~~

In [46]:
%%sqlite3
SELECT *
FROM tbl1
WHERE k0 NOT IN ('A', 'B') AND c13 NOT IN (200, 900)
ORDER BY c14;

[('E', 14, 832.44, 800, '2016-11-22', 0.39, 'EGFD'),
 ('E', 8, 302.86, 700, '2016-12-22', 0.14, 'DFCC'),
 ('E', 1, 273.08, 600, '2016-12-31', 0.21, 'BDGD'),
 ('E', 27, 720.9, 800, '2017-01-16', 0.12, 'FBGD'),
 ('D', 4, 662.69, 800, '2017-03-26', 0.23, 'BGDD'),
 ('E', 3, 305.43, 100, '2017-05-21', 0.21, 'BAED'),
 ('C', 13, 712.61, 400, '2017-10-23', 0.31, 'EDDA'),
 ('C', 5, 822.81, 100, '2017-11-17', 0.35, 'GGFC'),
 ('C', 7, 755.27, 800, '2018-07-04', 0.47, 'GCDB'),
 ('E', 25, 600.9, 700, '2018-11-07', 0.36, 'BBBA'),
 ('D', 2, 756.37, 500, '2019-02-28', 0.37, 'BCCC'),
 ('C', 19, 570.43, 400, '2019-04-12', 0.48, 'FBEE'),
 ('C', 24, 482.32, 300, '2019-05-03', 0.11, 'CCCE')]

### q08.sql

~~~
## 
## Escriba una consulta que compute el promedio
## de la columna c21 de la tabla tbl2 por ano 
## (columna c23) que este ordenada por año. 
##
~~~

In [63]:
%%sqlite3
SELECT strftime('%Y', c23) AS YEAR, AVG(c21) 
FROM tbl2
GROUP BY strftime('%Y', c23)
ORDER BY strftime('%Y', c23);

[('2016', 564.4764285714285),
 ('2017', 515.1563636363637),
 ('2018', 557.5593749999999),
 ('2019', 550.9985714285714)]